# RPA GESTIÓN DE PLATAFORMA

En este notebook se desarrolla un código que gestiona una plataforma, cambiando entre diferentes interfaces de la misma, pulsando en distintos botones en base a un listado de eventos que se introduce como input.

Incluye una recolección de capturas de pantalla para dejar evidencias del proceso, así como una parte de reconcimiento óptico de caracteres (OCR) para identificar las distintas variables para la toma de decisiones y los distintos caminos a seguir por la plataforma.

In [ ]:
# Importación de librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Carga de Datos

In [ ]:

ruta_archivo = "datos.csv"

# Carga del dataset
try:
    df = pd.read_csv(ruta_archivo)
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Archivo no encontrado. Verifica la ruta.")

## RPA_GEMAS_VAL_CLO.py

In [ ]:
#AUTOMATIZACION VALIDADO A CERRADO
import os #Necesaria para conseguir la ruta de un archivo
from datetime import datetime #Necesaria para el tratamiento de fechas
from selenium import webdriver
import time
import pandas as pd #Necesaria para el tratamiento de datos
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
# from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import warnings #Para desactivar los warnings
from selenium.webdriver.support.ui import Select
from datetime import datetime

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.reset_option('display.max_colwidth')
pd.set_option('display.width', 600)  # Ajusta el ancho a 200 caracteres
print("Libreriｭas importadas")

start_time_nat = time.time()  # guarda el tiempo actual
#SETTINGS

hoy = datetime.now().strftime('%d_%m_%Y')

ruta_importacion_excel = r"C:¥Users¥ES77850537E¥OneDrive - Enel Spa¥QOS_BI¥CﾓDIGOS MIGUEL¥RPA_GEMAS¥FICHEROS_EVENTOS¥PasarACerrado_v_2023_12_27.xlsx"
df_eventos = pd.read_excel(ruta_importacion_excel)
# df_eventos=df_eventos.head(5)
url = "http://gemas-es.enelint.global/#!/events"

chrome_options = webdriver.ChromeOptions()
#driver = webdriver.Chrome(executable_path='/ruta/a/chromedriver', options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)
#driver.maximize_window()

i=1
for index, fila in df_eventos.iterrows():
    evento = fila['EVENTOS']
    area = fila['AREA']
    inicio=time.time()
    current_time=datetime.now()
    formatted_time=current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # evento = '15241101'
    # area = 'ANDALUCIA'
    
    # chrome_options = webdriver.ChromeOptions()
    # driver = webdriver.Chrome(options=chrome_options)
    # time.sleep(1)
    
    print(f'{i} de {len(df_eventos)}----{formatted_time}')
    i+=1
    print(evento)
    print(area)
    driver.get(url)
    time.sleep(3)  # Esperamos que se cargue la pagina
    
    #####PARTE MANUAL#####
    
    # # body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select
    # print('dropdown')
    # # dropdown = driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
    # dropdown = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
    #     ))
    # select = Select(dropdown)
    # select.select_by_visible_text('CDS')

    # # dropdown = driver.find_element(By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
    # dropdown = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
    #     ))
    # time.sleep(1)
    # select = Select(dropdown)
    # time.sleep(1)
    # select.select_by_visible_text(area)
    # print('perfil gemas')
    # # boton_perfil_gemas=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
    # boton_perfil_gemas = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
    #    ))
    
    # boton_perfil_gemas.click()
    
    
    #####FIN PARTE MANUAL#####

    # time.sleep(1)
    print('gestion')
    # boton_gestion_eventos=driver.find_element(By.CSS_SELECTOR,'#manageEvent > a')
    boton_gestion_eventos = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
    boton_gestion_eventos.click()
    # time.sleep(1)
    print('caja evento')
    caja_evento=driver.find_element(By.CSS_SELECTOR,'#test > div.col-lg-9.col-md-8.col-sm-12.no-margin-bottom > div.form-group.no-margin-bottom.row.col-md-12.col-sm-12 > div:nth-child(1) > div.col-md-4.col-sm-4.col-xs-6 > input')  
    caja_evento.clear()
    caja_evento.send_keys(evento)
    # time.sleep(1)
    print('filtrar')
    # boton_filtrar=driver.find_element(By.CSS_SELECTOR, 'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
    boton_filtrar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
                                   ))
    boton_filtrar.click()
    # time.sleep(2)
    print('registro')
    
    # boton_registro = driver.find_element(By.XPATH,f"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")
    boton_registro = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")))
    boton_registro.click()    
    # time.sleep(2)
    try:
        # boton_cerrar=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
        boton_cerrar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')))
        boton_cerrar.click()
        # time.sleep(1)
        # boton_confirmar=driver.find_element(By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')
        boton_confirmar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')))
        boton_confirmar.click()
        # time.sleep(1)
        boton_guardar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'')))
        boton_guardar.click()
    except:
        print(f"Evento {evento} no puede ser cerrado, posiblemente ya esta asiｭ")
        continue
    
    #BOTﾓN VUELTA A LOS FILTROS?? SIN SALIR DEL PERFIL
    
    # driver.get(url)
    # driver.quit()
    
    
# Cerrar el navegador al finalizar
driver.quit()

print("Proceso de descarga completado.")

## RECORTE_OCR

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  4 23:28:41 2024

@author: Miguel Ángel Mayordomo Gragera
"""

from PIL import Image
import os

# Rutas de los directorios
directorio_origen = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres\captura_3_imagen"
directorio_destino = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres\captura_imagen_3_rec"

# directorio_origen = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\prueba"
# directorio_destino = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\prueba_"

# Dimensiones del recorte
ancho_recorte = 85  # Ancho del rectángulo de recorte
alto_recorte = 40  # Alto del rectángulo de recorte

# Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
inicio_x = 0.755  # 40% del ancho de la imagen
inicio_y = 0.33  # 50% del alto de la imagen

# Asegurarse de que el directorio de destino existe
if not os.path.exists(directorio_destino):
    os.makedirs(directorio_destino)

# Procesar cada archivo de imagen en el directorio
for filename in os.listdir(directorio_origen):
    print(filename)
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Ajusta según tus tipos de archivo
        ruta_completa = os.path.join(directorio_origen, filename)
        with Image.open(ruta_completa) as img:
            ancho, alto = img.size

            # Calcula el punto de inicio del recorte
            left = int(ancho * inicio_x)
            top = int(alto * inicio_y)

            # Asegúrate de que el recorte no exceda las dimensiones de la imagen
            right = min(left + ancho_recorte, ancho)
            bottom = min(top + alto_recorte, alto)

            img_recortada = img.crop((left, top, right, bottom))
            img_recortada.save(os.path.join(directorio_destino, "recortada_" + filename))


#RECONOCIMIENTO CARACTERES
import pytesseract
from PIL import Image
import os
import pandas as pd


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


directorio_destino = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres\captura_imagen_3_rec"

# directorio_destino = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\prueba_"

# Lista para almacenar los resultados
resultados = []

# Procesar cada archivo de imagen en el directorio
i=0
for filename in os.listdir(directorio_destino):
    i+=1
    print(i)
    print(filename)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        ruta_completa = os.path.join(directorio_destino, filename)
        with Image.open(ruta_completa) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            print(texto)
            resultados.append({'Nombre del Archivo': filename, 'Texto': texto})

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados)

df['Texto'] = df['Texto'].astype(str)

# Crear un nuevo DataFrame con los nombres de las imágenes cuyo texto no sea "Cerrado"
# df_no_cerrado = df.loc[df['Texto'].str.replace(" ", "").replace("\n", "").str.lower() != 'cerrado']
df_no_cerrado = df.loc[~df['Texto'].str.replace(" ", "").replace("\n", "").str.lower().str.contains('cerr(ado)?')]


df_no_cerrado['Codigo'] = df_no_cerrado['Nombre del Archivo'].str.extract(r'recortada_(\d+)_\d+\.png')

df_codigos = df_no_cerrado[['Codigo']]


df_codigos.to_excel(r'C:\Users\migue\OneDrive - CIBERNOS\Escritorio\prueba_\codigos.xlsx', index=False)


## RPA_citrix_v2

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 17 07:54:52 2024

@author: Miguel Ángel Mayordomo Gragera
"""

#______________________________________________COMIENZO MANUAL_______________________________________
#Preparar el excel de eventos. Un excel por cada área.
#Entrar en citrix
#Poner ventana en modo pantalla completa
#Entrar en GDS - GEMAS
#Seleccion perfil usuario CDS
#Seleccion territorio
#Pulsar botón Perfil GEMAS

###HECHO PARA 3 PANTALLAS Y EL ORDENADOR A LA IZQUIERDA, COMPROBAR CON OTRAS MODIFICACIONES DE LAS POSICIONES DE LAS PANTALLAS

#AUTOMATIZACION VALIDADO A CERRADO PARA CITRIX
import os #Necesaria para conseguir la ruta de un archivo
import time
import pandas as pd #Necesaria para el tratamiento de datos
import warnings #Para desactivar los warnings
from datetime import datetime, timedelta
import pyautogui
from PIL import Image
import pytesseract

#PARÁMETROS
area='ANDALUCIA'

ruta_tesseract=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
ruta_importacion_excel = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\Pasar_a_cerrado.xlsx"
ruta_capturas='C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/'
ruta_loading=f"{ruta_capturas}LOADING/load.png"
ruta_estado_cerrado=f"{ruta_capturas}LOADING/cerrado.png"

#INICIO DEL PROGRAMA
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"Inicio a las {formatted_time} para {area}")

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
print("Libreri­as importadas")

def detecta_loading (ruta): #Función para detectar cuando la página está cargando (loading)
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    ancho_recorte = 200  # Ancho del rectángulo de recorte
    alto_recorte = 50  # Alto del rectángulo de recorte

    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.47  # 50% del ancho de la imagen
    inicio_y = 0.495  # 50% del alto de la imagen
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = ruta_tesseract
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto
    
def detecta_error (ruta): #Función para detectar un error. 
    #Lo que hace es una captura de la zona de la pantalla que pone "Cerrado". Si no pone "Cerrado" habrá un error en ese evento.
    captura = pyautogui.screenshot()
    captura.save(ruta)
    ancho_recorte = 85  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.755  # 40% del ancho de la imagen
    inicio_y = 0.33  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = ruta_tesseract
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            
        return texto

def detecta_error_web (ruta): #Función para detectar cuando la página de gestión de eventos no ha cargado bien.
    #El error se detecta cuando en la captura de pantalla no aparece la palabra "temporal" en una zona de la captura, indicando que se ha entrado en otro apartado por error.
    captura = pyautogui.screenshot()
    captura.save(ruta)
    ancho_recorte = 80  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.105  # 40% del ancho de la imagen
    inicio_y = 0.3  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = ruta_tesseract
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto

start_time_nat = time.time()  # guarda el tiempo actual

hoy = datetime.now().strftime('%d_%m_%Y')
df_eventos = pd.read_excel(ruta_importacion_excel,sheet_name='EventosParaCerrar',dtype=str)
df_eventos.rename(columns={df_eventos.columns[0]: 'EVENTOS'}, inplace=True)

print("Datos importados")

eventos_ejecutados = pd.DataFrame(columns=['Evento'])
eventos_error = pd.DataFrame(columns=['Evento'])

#SI SE CORTASE EL PROGRAMA, HABRÍA QUE EJECUTARLO DESDE AQUÍ HACIA ABAJO PARA QUE NO SE REPITAN LOS MISMOS EVENTOS QUE YA SE HAN EJECUTADO
df_eventos = df_eventos[~df_eventos['EVENTOS'].isin(eventos_ejecutados['Evento'])]
eventos_error.rename(columns={eventos_error.columns[0]: 'EVENTOS'}, inplace=True)

i=0
start_time = datetime.now()
end_time = datetime.now()

for index, fila in df_eventos.iterrows():
    evento = fila['EVENTOS']
    # evento = '15241101' #Evento para probar
    elapsed_time = (end_time - current_time).total_seconds()
    
    current_time = datetime.now()
    print(f"Ha tardado {elapsed_time}")

    # Estima el tiempo restante
    events_left = len(df_eventos) - (i + 1)
    estimated_time_remaining = elapsed_time * events_left
    
    # Calcula la hora prevista de finalización
    estimated_end_time = current_time + timedelta(seconds=estimated_time_remaining)
    
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    estimated_end_formatted = estimated_end_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # Formatear el tiempo estimado restante en horas y minutos
    hours_remaining = int(estimated_time_remaining // 3600)
    minutes_remaining = int((estimated_time_remaining % 3600) // 60)
   
    print(f'{evento}--{i + 1} de {len(df_eventos)}--faltan {hours_remaining} horas y {minutes_remaining} minutos. Fin a las {estimated_end_formatted}')
    
#___#Pulsar botón Gestión de Eventos
    pyautogui.click(595,317,button='left')

    time.sleep(0.5)
    
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    estado_web=detecta_error_web(ruta_estado_cerrado)
    print(estado_web)
    
    if "temporal" not in estado_web:
        nuevo_evento_df = pd.DataFrame({'Evento': [evento]})
        print(f"Error en el evento {evento}, se pasa al siguiente")
        eventos_error = pd.concat([eventos_error, nuevo_evento_df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue
    
#___#Pulsar en la caja de evento
    pyautogui.click(405,407,button='left')
    pyautogui.click(405,407,button='left')
    time.sleep(1)
    
#___#Escribir event_cod
    captura = pyautogui.screenshot()
    captura.save(f"{ruta_capturas}{evento}.png")
    pyautogui.click(405,407,button='left')
    pyautogui.write(f'{evento}')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
        
    captura1 = pyautogui.screenshot()
    captura1.save(f"{ruta_capturas}{evento}_1.png")

#___#Pulsar en Filtrar
    pyautogui.click(1323,260,button='left')

    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)

#___#Pulsar en Registro
    pyautogui.click(850,376,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
#___#Pulsar en Cerrar
    pyautogui.click(1079,265,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
#___#Pulsar en Confirmar
    pyautogui.click(1436,310,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
#___#Pulsar en Cerrar dialog
    pyautogui.click(1570,299,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
#___#Pulsar en Guardar
    pyautogui.click(1890,413,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    pyautogui.click(1890,413,button='left')
    captura2 = pyautogui.screenshot()
    captura2.save(f"{ruta_capturas}{evento}_2.png")
    captura2.save(f"{ruta_capturas}{evento}_2.png")
    time.sleep(1)
        
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.3)
    
    estado_final=detecta_error(ruta_estado_cerrado)
    print(estado_final)
    
    nuevo_evento_df = pd.DataFrame({'Evento': [evento]})
    
    if "Cerrado" not in estado_final:
        print(f"Error en el evento {evento}, se pasa al siguiente")
        eventos_error = pd.concat([eventos_error, nuevo_evento_df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue
    
#___#Pulsar en Inicio
    pyautogui.click(57,207,button='left')
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    i+=1
    eventos_ejecutados = pd.concat([eventos_ejecutados, nuevo_evento_df], ignore_index=True)

    end_time = datetime.now()

print("Proceso completado.")
print("Eventos con errores:")
print(f'{eventos_error}')

#Si el proceso ha terminado y hay eventos con errores hay que repetir el proceso pero haciendo df_eventos=eventos_error.copy() encima del bucle

#Al finalizar eliminar las capturas para no ocupar espacio innecesario








#CÓDIGO AUXILIAR________________________________________________________________________________________________
# x, y = pyautogui.position()
# print(x, y)

# i=0
# while i<100:
#     x, y = pyautogui.position()
#     print(x, y)
#     i+=1
#     time.sleep(2)


# import os
# import shutil

# # Rutas de las carpetas
# ruta_origen = r'C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres'
# ruta_destino = r'C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres\captura_3_imagen'

# # Verificando si la ruta destino existe, si no, la creamos
# if not os.path.exists(ruta_destino):
#     os.makedirs(ruta_destino)

# # Recorrer los archivos en la carpeta de origen
# for archivo in os.listdir(ruta_origen):
#     # Verificar si el archivo termina en '_2.png'
#     if archivo.endswith('_2.png'):
#         # Ruta completa del archivo origen y destino
#         archivo_origen = os.path.join(ruta_origen, archivo)
#         archivo_destino = os.path.join(ruta_destino, archivo)

#         # Copiar el archivo
#         shutil.copy(archivo_origen, archivo_destino)
#         print(f"Archivo {archivo} copiado con éxito.")

# print("Proceso de copia finalizado.")

## RPA_GEMS_VAL_CLO_citrix.py

In [ ]:
#AUTOMATIZACION VALIDADO A CERRADO PARA CITRIX
import os #Necesaria para conseguir la ruta de un archivo
from datetime import datetime #Necesaria para el tratamiento de fechas
# from selenium import webdriver
import time
import pandas as pd #Necesaria para el tratamiento de datos
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import requests
# from selenium.common.exceptions import ElementClickInterceptedException
# from bs4 import BeautifulSoup
import warnings #Para desactivar los warnings
# from selenium.webdriver.support.ui import Select
from datetime import datetime, timedelta

import pyautogui
from PIL import Image
import pytesseract
# import requests

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.reset_option('display.max_colwidth')
pd.set_option('display.width', 600)  # Ajusta el ancho a 200 caracteres
print("Libreri­as importadas")

def detecta_loading (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    ancho_recorte = 200  # Ancho del rectángulo de recorte
    alto_recorte = 50  # Alto del rectángulo de recorte

    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.47  # 50% del ancho de la imagen
    inicio_y = 0.495  # 50% del alto de la imagen
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto
    
def detecta_error (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    # Dimensiones del recorte
    ancho_recorte = 85  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.755  # 40% del ancho de la imagen
    inicio_y = 0.33  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto

def detecta_error_web (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    # Dimensiones del recorte
    ancho_recorte = 80  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.105  # 40% del ancho de la imagen
    inicio_y = 0.3  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto

start_time_nat = time.time()  # guarda el tiempo actual
#SETTINGS
area='ANDALUCIA'
hoy = datetime.now().strftime('%d_%m_%Y')

#Ruta Aníbal
ruta_importacion_excel = r"C:\Users\ES77850537E\OneDrive - Enel Spa\QOS_BI\CÓDIGOS MIGUEL\RPA_GEMAS\FICHEROS_EVENTOS\PasarACerrado_v_2023_12_27_or.xlsx"
#Ruta Miguel
ruta_importacion_excel = r"C:\Users\migue\Enel Spa\Vargas Castejon, Antonio Manuel - CÓDIGOS MIGUEL\RPA_GEMAS\FICHEROS_EVENTOS\PasarACerrado_v_2023_12_27_or.xlsx"
ruta_importacion_excel = r"C:\Users\migue\OneDrive - CIBERNOS\Escritorio\Pasar_a_cerrado.xlsx"

df_eventos = pd.read_excel(ruta_importacion_excel,sheet_name='Hoja1',dtype=str)

df_eventos.rename(columns={df_eventos.columns[0]: 'EVENTOS'}, inplace=True)

df_eventos_invertido = df_eventos.iloc[::-1]
df_eventos=df_eventos_invertido.copy()

# df_eventos=df_eventos.head(10)
# df_eventos = df_eventos.iloc[10:20]
print("Datos importados")

###APERTURA MANUAL DE GEMAS EN CITRIX. HAY QUE PONER LA VENTANA EN MODO PANTALLA COMPLETA. 
###HECHO PARA 3 PANTALLAS Y EL ORDENADOR A LA IZQUIERDA, COMPROBAR CON OTRAS MODIFICACIONES DE LAS POSICIONES DE LAS PANTALLAS
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"Inicio a las {formatted_time}")
#Seleccion perfil usuario CDS
pyautogui.click(316,222,button='left')
time.sleep(1)
pyautogui.click(294,260,button='left')
time.sleep(1)
#Seleccion territorio
pyautogui.click(368,258,button='left')
time.sleep(1)
if area == 'ANDALUCIA':
    pyautogui.click(338,298,button='left')
elif area == 'BALEARES':
    pyautogui.click(338,317,button='left')
elif area == 'CANARIAS':
    pyautogui.click(338,338,button='left')
elif area == 'CATALUNYA':
    pyautogui.click(338,362,button='left')
elif area == 'ARAGON':
    pyautogui.click(338,382,button='left')
else:
    print("Area no seleccionada")
time.sleep(1)
#Pulsar botón Perfil GEMAS
pyautogui.click(343,331,button='left')
time.sleep(3) #Más tiempo por cambio de página


eventos_ejecutados = pd.DataFrame(columns=['Evento'])
eventos_error = pd.DataFrame(columns=['Evento'])
# Eliminar los últimos 75 registros
# copia_seg=eventos_ejecutados.copy()

# eventos_ejecutados=copia_seg.copy()

# eventos_ejecutados = eventos_ejecutados.iloc[:-75]

df_eventos = df_eventos[~df_eventos['EVENTOS'].isin(eventos_ejecutados['Evento'])]

ruta_loading="C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/LOADING/load.png"
ruta_estado_cerrado="C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/LOADING/cerrado.png"


eventos_error.rename(columns={eventos_error.columns[0]: 'EVENTOS'}, inplace=True)
df_eventos=eventos_error.copy()

print(area)
i=0
start_time = datetime.now()
end_time = datetime.now()

for index, fila in df_eventos.iterrows():
    evento = fila['EVENTOS']
    # evento = '15241101'
    elapsed_time = (end_time - current_time).total_seconds()
    
    current_time = datetime.now()
    print(f"Ha tardado {elapsed_time}")

    # Estima el tiempo restante
    events_left = len(df_eventos) - (i + 1)
    estimated_time_remaining = elapsed_time * events_left
    
    # Calcula la hora prevista de finalización
    estimated_end_time = current_time + timedelta(seconds=estimated_time_remaining)
    
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    estimated_end_formatted = estimated_end_time.strftime("%Y-%m-%d %H:%M:%S")
    # Formatear el tiempo estimado restante en horas y minutos
    hours_remaining = int(estimated_time_remaining // 3600)
    minutes_remaining = int((estimated_time_remaining % 3600) // 60)
   
    print(f'{evento}--{i + 1} de {len(df_eventos)}--faltan {hours_remaining} horas y {minutes_remaining} minutos. Fin a las {estimated_end_formatted}')
    #Pulsar botón Gestión de Eventos
    pyautogui.click(595,317,button='left')

    time.sleep(0.5)
    
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    estado_web=detecta_error_web(ruta_estado_cerrado)
    print(estado_web)
    
    if "temporal" not in estado_web:
        nuevo_evento_df = pd.DataFrame({'Evento': [evento]})
        print(f"Error en el evento {evento}, se pasa al siguiente")
        eventos_error = pd.concat([eventos_error, nuevo_evento_df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue


    pyautogui.click(405,407,button='left')
    pyautogui.click(405,407,button='left')
    time.sleep(1)
    #Escribir event_cod
    captura = pyautogui.screenshot()
    captura.save(f"C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/{evento}.png")
    pyautogui.click(405,407,button='left')
    pyautogui.write(f'{evento}')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
        
    captura1 = pyautogui.screenshot()
    captura1.save(f"C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/{evento}_1.png")

    #Pulsar en Filtrar
    pyautogui.click(1288,264,button='left')

    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)

    #Pulsar en Registro
    pyautogui.click(850,376,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Cerrar
    pyautogui.click(1079,265,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Confirmar
    pyautogui.click(1436,310,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Cerrar dialog
    pyautogui.click(1570,299,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Guardar
    pyautogui.click(1890,413,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    pyautogui.click(1890,413,button='left')
    captura2 = pyautogui.screenshot()
    captura2.save(f"C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/{evento}_2.png")
    captura2.save(f"C:/Users/migue/OneDrive - CIBERNOS/Escritorio/capturas_eventos_cierres/captura_3_imagen/{evento}_2.png")
    time.sleep(1)
        
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.3)
    
    estado_final=detecta_error(ruta_estado_cerrado)
    print(estado_final)
    
    nuevo_evento_df = pd.DataFrame({'Evento': [evento]})
    
    if "Cerrado" not in estado_final:
        print(f"Error en el evento {evento}, se pasa al siguiente")
        eventos_error = pd.concat([eventos_error, nuevo_evento_df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue
    
    #Pulsar en Inicio
    pyautogui.click(57,207,button='left')
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    i+=1
    eventos_ejecutados = pd.concat([eventos_ejecutados, nuevo_evento_df], ignore_index=True)

    end_time = datetime.now()

print("Proceso completado.")

#AUXILIAR
# x, y = pyautogui.position()
# print(x, y)

# i=0
# while i<100:
#     x, y = pyautogui.position()
#     print(x, y)
#     i+=1
#     time.sleep(2)


# import os
# import shutil

# # Rutas de las carpetas
# ruta_origen = r'C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres'
# ruta_destino = r'C:\Users\migue\OneDrive - CIBERNOS\Escritorio\capturas_eventos_cierres\captura_3_imagen'

# # Verificando si la ruta destino existe, si no, la creamos
# if not os.path.exists(ruta_destino):
#     os.makedirs(ruta_destino)

# # Recorrer los archivos en la carpeta de origen
# for archivo in os.listdir(ruta_origen):
#     # Verificar si el archivo termina en '_2.png'
#     if archivo.endswith('_2.png'):
#         # Ruta completa del archivo origen y destino
#         archivo_origen = os.path.join(ruta_origen, archivo)
#         archivo_destino = os.path.join(ruta_destino, archivo)

#         # Copiar el archivo
#         shutil.copy(archivo_origen, archivo_destino)
#         print(f"Archivo {archivo} copiado con éxito.")

# print("Proceso de copia finalizado.")





## RPA_GEMS_VAL_CLO_v2.py

In [ ]:
#AUTOMATIZACION VALIDADO A CERRADO
import os #Necesaria para conseguir la ruta de un archivo
from datetime import datetime #Necesaria para el tratamiento de fechas
from selenium import webdriver
import time
import pandas as pd #Necesaria para el tratamiento de datos
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
# from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import warnings #Para desactivar los warnings
from selenium.webdriver.support.ui import Select
from datetime import datetime

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.reset_option('display.max_colwidth')
pd.set_option('display.width', 600)  # Ajusta el ancho a 200 caracteres
print("Libreri­as importadas")

start_time_nat = time.time()  # guarda el tiempo actual
#SETTINGS
area='ANDALUCIA'
hoy = datetime.now().strftime('%d_%m_%Y')

ruta_importacion_excel = r"C:\Users\ES77850537E\OneDrive - Enel Spa\QOS_BI\CÓDIGOS MIGUEL\RPA_GEMAS\FICHEROS_EVENTOS\Pasar_a_cerrado_24_01.xlsx"
df_eventos = pd.read_excel(ruta_importacion_excel)
# df_eventos=df_eventos.head(10)
# df_eventos = df_eventos.iloc[5:10]
print("Datos importados")
url = "http://gemas-es.enelint.global/#!/events"

chrome_options = webdriver.ChromeOptions()
#driver = webdriver.Chrome(executable_path='/ruta/a/chromedriver', options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)
#driver.maximize_window()
driver.get(url)
time.sleep(3)  # Esperamos que se cargue la pagina
i=1

#####PARTE INICIO#####

# body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select
print('dropdown')
# dropdown = driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
dropdown = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
    ))
select = Select(dropdown)
select.select_by_visible_text('CDS')

# dropdown = driver.find_element(By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
dropdown = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
    ))
time.sleep(1)
select = Select(dropdown)
time.sleep(1)
select.select_by_visible_text(area)
print('perfil gemas')
# boton_perfil_gemas=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
boton_perfil_gemas = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
    ))

boton_perfil_gemas.click()

# time.sleep(1)
print('gestion')
# boton_gestion_eventos=driver.find_element(By.CSS_SELECTOR,'#manageEvent > a')
boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
boton_gestion_eventos.click()
# time.sleep(1)

#####FIN PARTE MANUAL#####
print(area)
for index, fila in df_eventos.iterrows():
    evento = fila['EVENTOS']
    area = fila['AREA']
    inicio=time.time()
    current_time=datetime.now()
    formatted_time=current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # evento = '15241101'
    # area = 'ANDALUCIA'
    
    # chrome_options = webdriver.ChromeOptions()
    # driver = webdriver.Chrome(options=chrome_options)
    # time.sleep(1)
    
    print(f'{i} de {len(df_eventos)}----{formatted_time}')
    i+=1
    print(evento)
    

    print('caja evento')
    
    # caja_evento=driver.find_element(By.CSS_SELECTOR,'#test > div.col-lg-9.col-md-8.col-sm-12.no-margin-bottom > div.form-group.no-margin-bottom.row.col-md-12.col-sm-12 > div:nth-child(1) > div.col-md-4.col-sm-4.col-xs-6 > input')  
    caja_evento = WebDriverWait(driver, 300).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'#test > div.col-lg-9.col-md-8.col-sm-12.no-margin-bottom > div.form-group.no-margin-bottom.row.col-md-12.col-sm-12 > div:nth-child(1) > div.col-md-4.col-sm-4.col-xs-6 > input')
                                   ))
    caja_evento.clear()
    caja_evento.send_keys(evento)
    # time.sleep(1)
    print('filtrar')
    # boton_filtrar=driver.find_element(By.CSS_SELECTOR, 'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
    boton_filtrar = WebDriverWait(driver, 300).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
                                   ))
    boton_filtrar.click()
    # time.sleep(2)
    print('registro')
    
    # boton_registro = driver.find_element(By.XPATH,f"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")
    boton_registro = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")))
    boton_registro.click()    
    # time.sleep(2)
    try:
        # boton_cerrar=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
        boton_cerrar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')))
        boton_cerrar.click()
        # time.sleep(1)
        # boton_confirmar=driver.find_element(By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')
        boton_confirmar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')))
        boton_confirmar.click()
        time.sleep(2)
        boton_cerrar_dial = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button')))
        boton_cerrar_dial.click()
        
       
        time.sleep(1)
        boton_guardar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#listEvents-eye > div:nth-child(2) > button')))
        boton_guardar.click()
        time.sleep(1)
        
        boton_inicio = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div:nth-child(1) > div > div > button > span')))
        boton_inicio.click()
        
        
        boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
        boton_gestion_eventos.click()
        # body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button
        print("Se ha cerrado el evento con éxito")
    except:
        print("Evento ya cerrado de antes")
            
        boton_inicio = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div:nth-child(1) > div > div > button > span')))
        boton_inicio.click()

        boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
        boton_gestion_eventos.click()
        
        continue
    time.sleep(1)
    #BOTÓN VUELTA A LOS FILTROS?? SIN SALIR DEL PERFIL

    # boton_lupa = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div.row.wrapper > div > div.col-xs-1.col-sm-1.col-md-1.col-lg-1.nav-search-page > ul > li.tabs-right-search > a')))
    # boton_lupa.click()

   
    # driver.get(url)
    # driver.quit()
    
    
# Cerrar el navegador al finalizar
driver.quit()

print("Proceso de descarga completado.")

## RPA_GEMS_VAL_CLO_v3.py

In [ ]:
#AUTOMATIZACION VALIDADO A CERRADO
import os #Necesaria para conseguir la ruta de un archivo
from datetime import datetime #Necesaria para el tratamiento de fechas
from selenium import webdriver
import time
import pandas as pd #Necesaria para el tratamiento de datos
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
# from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import warnings #Para desactivar los warnings
from selenium.webdriver.support.ui import Select
from datetime import datetime

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.reset_option('display.max_colwidth')
pd.set_option('display.width', 600)  # Ajusta el ancho a 200 caracteres
print("Libreri­as importadas")

start_time_nat = time.time()  # guarda el tiempo actual
#SETTINGS
area='ANDALUCIA'
hoy = datetime.now().strftime('%d_%m_%Y')

ruta_importacion_excel = r"C:\Users\ES77850537E\OneDrive - Enel Spa\QOS_BI\CÓDIGOS MIGUEL\RPA_GEMAS\FICHEROS_EVENTOS\PasarACerrado_v_2023_12_27_or.xlsx"
df_eventos = pd.read_excel(ruta_importacion_excel)
df_eventos=df_eventos.head(10)
# df_eventos = df_eventos.iloc[5:10]
print("Datos importados")
url = "http://gemas-es.enelint.global/#!/events"

chrome_options = webdriver.ChromeOptions()
#driver = webdriver.Chrome(executable_path='/ruta/a/chromedriver', options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)
#driver.maximize_window()
driver.get(url)
time.sleep(3)  # Esperamos que se cargue la pagina
i=1

#####PARTE INICIO#####

# body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select
print('dropdown')
# dropdown = driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
dropdown = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(1) > div:nth-child(2) > select')
    ))
select = Select(dropdown)
select.select_by_visible_text('CDS')

# dropdown = driver.find_element(By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
dropdown = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(2) > div:nth-child(2) > select')
    ))
time.sleep(1)
select = Select(dropdown)
time.sleep(1)
select.select_by_visible_text(area)
print('perfil gemas')
# boton_perfil_gemas=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
boton_perfil_gemas = WebDriverWait(driver, 300).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div.col-xs-12.col-sm-12.col-md-12.col-lg-12.profileSelectionContainer > form > div > div:nth-child(4) > div:nth-child(2) > input')
    ))

boton_perfil_gemas.click()

# time.sleep(1)
print('gestion')
# boton_gestion_eventos=driver.find_element(By.CSS_SELECTOR,'#manageEvent > a')
boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
boton_gestion_eventos.click()
# time.sleep(1)

df_eventos_error = pd.DataFrame(columns=['Evento', 'Error'])

#####FIN PARTE MANUAL#####
print(area)
for index, fila in df_eventos.iterrows():
    time.sleep(1)
    evento = fila['EVENTOS']
    area = fila['AREA']
    inicio=time.time()
    current_time=datetime.now()
    formatted_time=current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # evento = '15241101'
    # area = 'ANDALUCIA'
    
    # chrome_options = webdriver.ChromeOptions()
    # driver = webdriver.Chrome(options=chrome_options)
    # time.sleep(1)
    try:
        print(f'{i} de {len(df_eventos)}----{formatted_time}')
        i+=1
        print(evento)

        print('caja evento')
       
        # caja_evento=driver.find_element(By.CSS_SELECTOR,'#test > div.col-lg-9.col-md-8.col-sm-12.no-margin-bottom > div.form-group.no-margin-bottom.row.col-md-12.col-sm-12 > div:nth-child(1) > div.col-md-4.col-sm-4.col-xs-6 > input')  
        caja_evento = WebDriverWait(driver, 300).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'#test > div.col-lg-9.col-md-8.col-sm-12.no-margin-bottom > div.form-group.no-margin-bottom.row.col-md-12.col-sm-12 > div:nth-child(1) > div.col-md-4.col-sm-4.col-xs-6 > input')
                                       ))
        caja_evento.clear()
        caja_evento.send_keys(evento)
        # time.sleep(1)
        print('filtrar')
        # boton_filtrar=driver.find_element(By.CSS_SELECTOR, 'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
        boton_filtrar = WebDriverWait(driver, 300).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
                                       ))
        boton_filtrar.click()
        # time.sleep(2)
        print('registro')
        
        # boton_registro = driver.find_element(By.XPATH,f"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")
        boton_registro = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[3]/div/div/div[4]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div/div/div[2]/div")))
        boton_registro.click()    
        # time.sleep(2)
        try:
            # boton_cerrar=driver.find_element(By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')
            boton_cerrar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.col-sm-12.container > div > div > div:nth-child(2) > div > toolbar > div > div.col-md-6.col-xs-6.col-sm-6.ng-scope > ul > li:nth-child(1) > button')))
            boton_cerrar.click()
            # time.sleep(1)
            # boton_confirmar=driver.find_element(By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')
            boton_confirmar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button.btn.btn-primary.ng-binding')))
            boton_confirmar.click()
            time.sleep(2)
            boton_cerrar_dial = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button')))
            boton_cerrar_dial.click()
            
           
            time.sleep(1)
            boton_guardar = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#listEvents-eye > div:nth-child(2) > button')))
            boton_guardar.click()
            time.sleep(1)
            
            boton_inicio = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div:nth-child(1) > div > div > button > span')))
            boton_inicio.click()
            
            
            boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
            boton_gestion_eventos.click()
            # body > div.modal.ng-isolate-scope.in > div > div > div > div.modal-footer > button
            print("Se ha cerrado el evento con éxito")
        except:
            print("Evento ya cerrado de antes")
                
            boton_inicio = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div:nth-child(1) > div > div > button > span')))
            boton_inicio.click()
    
            boton_gestion_eventos = WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#manageEvent > a')))
            boton_gestion_eventos.click()
            
    except Exception as e:
        print(f"Ha sucedido algún error con el evento {evento}")
        df_eventos_error = df_eventos_error.append({'Evento': evento, 'Error': str(e)}, ignore_index=True)
        continue
    time.sleep(1)
    

driver.quit()
print("Errores encontrados:")
print(df_eventos_error)    

print("Proceso de descarga completado.")


#CÓDIGO AUXILIAR
#BOTÓN VUELTA A LOS FILTROS?? SIN SALIR DEL PERFIL

# boton_lupa = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR,' body > div.col-sm-12.container > div > div > div.row.wrapper > div > div.col-xs-1.col-sm-1.col-md-1.col-lg-1.nav-search-page > ul > li.tabs-right-search > a')))
# boton_lupa.click()

   
# driver.get(url)
# driver.quit()